In [1]:
import numpy as np
import torch
from modeling.deeplab import DeepLab
from torchvision.transforms import transforms
from PIL import Image

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
backbone='resnet'  #resnet
PATH='checkpoint.pth.tar'
model = DeepLab(num_classes=2,
                        backbone=backbone,
                        output_stride=16,
                        sync_bn=False
                        ).to(DEVICE)
checkpoint = torch.load(PATH)

model.load_state_dict(checkpoint['state_dict'])
model.eval()

G:\Anaconda\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DeepLab(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
         

In [25]:
transform_img = transforms.Compose([
        transforms.Resize((80, 100), interpolation=Image.BILINEAR),
        transforms.ToTensor(),
    ])

img_path='./weizmann_horse_db/horse_test/horse282.png'
input=Image.open(img_path).convert('RGB')
input=transform_img(input)
input=input.reshape(1,3,80,100)

input=input.to(torch.float32)
with torch.no_grad():
    input=input.to(DEVICE)
    output = model(input)

pred = output.data.cpu().numpy()
pred = np.argmax(pred, axis=1)
pred = pred[0]
pred=pred*255
image = Image.fromarray(pred.astype(np.uint8))
image.show()

In [6]:
from PIL import Image
import numpy as np
mask_path='./weizmann_horse_db/mask_test/horse282.png'
mask = Image.open(mask_path)
mask = np.array(mask)
mask = mask*255
mask = Image.fromarray(mask.astype(np.uint8))
mask.show()